In [1]:
from langchain import LLMChain, PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [3]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0)

In [4]:
template = """
Answer the question based on the context given below. If the question can not be answered with the provided context then answer 
with I dont know. 

Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.

question: {question}
answer: 
"""

prompt = PromptTemplate(template= template, input_variables=['question'])

In [5]:
chain = LLMChain(prompt= prompt, llm=llm)

In [6]:
input_data1 = {"question": "What is quantum computing?"}
input_data2 = {"question": "What is World highest moutain?"}

response1 = chain.run(input_data1)
response2 = chain.run(input_data2)

In [7]:
print(response1)

Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.


In [8]:
print(response2)

I don't know. The provided context does not contain any information about the world's highest mountain.


# Creating a Scarcastic Ai Assistant

In [10]:
from langchain import FewShotPromptTemplate

examples = [
    {
        "query": "How do I become a better programmer?",
        'answer': "Try talking to a rubber duck; it works wonders."
    },
    {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    template= example_template,
    input_variables = ['query', 'answer']
)

prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to users' questions. Here are some
examples:"""

suffix = """User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=['query'],
    example_separator="\n\n"
)

In [11]:
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

In [12]:
input_data = {"query": "How can I learn quantum computing?"}
response = chain.run(input_data)

In [13]:
print(response)

By reading tea leaves and staring at a crystal ball.


# Alternating Human/AI messages

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

In [5]:
chat_llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0, convert_system_message_to_human=True)

In [6]:
template = "You are a helpful assistant that translates the english language to pirate language"
system_message_prompt = SystemMessagePromptTemplate.from_template(template=template)

example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")

human_message = '{text}'
human_message_prompt = HumanMessagePromptTemplate.from_template(human_message)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,
                                                example_human,
                                                example_ai,
                                                human_message_prompt
])

chain = LLMChain(llm=chat_llm, prompt=chat_prompt)

In [9]:
chain.run("I love dogs.")


StopCandidateException: index: 0
content {
  parts {
    text: "Ahoy there, ye landlubber! Ye be speakin\' of yer love fer dogs, be ye? Well, let me tell ye, there be nothin\' finer than a loyal pooch by yer side. Dogs be man\'s best matey, and they\'ll be there fer ye through thick and thin. They\'ll wag their tails and lick yer face, and they\'ll never ask fer nothin\' in return. So if ye be lookin\' fer a true friend, get yerself a dog, me hearty!"
  }
  role: "model"
}
finish_reason: SAFETY
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: LOW
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: HIGH
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}


# Example Selector

In [10]:
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain import FewShotPromptTemplate, PromptTemplate

In [11]:
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

example_template = """Word: {word}
Antonym: {antonym}"""

example_prompt = PromptTemplate(template=example_template, input_variables=['word', 'antonym'])

In [12]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=25,
)

In [13]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "Give the antonym of every input",
    suffix = 'Word: {input}\nAntonym:',
    input_variables=["input"],
    example_separator="\n\n",
)

In [14]:
print(dynamic_prompt.format(input="big"))

Give the antonym of every input

Word: happy
Antonym: sad

Word: tall
Antonym: short

Word: energetic
Antonym: lethargic

Word: sunny
Antonym: gloomy

Word: windy
Antonym: calm

Word: big
Antonym:


# Semantics Similarity Example Selector

In [15]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import DeepLake
from langchain.embeddings import GooglePalmEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

c:\Users\dell\OneDrive\Documents\LangChain and Vector database\langchain\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.13) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [16]:
# Define some examples
examples = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

# Create a PromptTemplate
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

In [17]:
# create Deep Lake dataset
my_activeloop_org_id = "samman" 
my_activeloop_dataset_name = "langchain_course_fewshot_selector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path)

Your Deep Lake dataset has been successfully created!


In [18]:
# Embedding function
embeddings = GooglePalmEmbeddings()

In [19]:
# Instantiate SemanticSimilarityExampleSelector using the examples
example_selector = SemanticSimilarityExampleSelector.from_examples(examples=examples, embeddings=embeddings, vectorstore_cls=db, k=1)

Creating 5 embeddings in 1 batches of size 5:: 100%|██████████| 1/1 [00:04<00:00,  4.72s/it]

Dataset(path='./deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype     shape     dtype  compression
  -------    -------   -------   -------  ------- 
   text       text      (5, 1)     str     None   
 metadata     json      (5, 1)     str     None   
 embedding  embedding  (5, 768)  float32   None   
    id        text      (5, 1)     str     None   


In [20]:
# Create a FewShotPromptTemplate using the example_selector
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Convert the temperature from Celsius to Fahrenheit",
    suffix="Input: {temperature}\nOutput:",
    input_variables=['temperature']
)

In [22]:
 #Test the similar_prompt with different inputs
print(similar_prompt.format(temperature="10°C"))   # Test with an input
print(similar_prompt.format(temperature="30°C"))  # Test with another input

Convert the temperature from Celsius to Fahrenheit

Input: 10°C
Output: 50°F

Input: 10°C
Output:
Convert the temperature from Celsius to Fahrenheit

Input: 30°C
Output: 86°F

Input: 30°C
Output:


In [23]:
# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print(similar_prompt.format(temperature="40°C")) # Test with a new input after adding the example

Creating 1 embeddings in 1 batches of size 1:: 100%|██████████| 1/1 [00:00<00:00,  2.09it/s]


Dataset(path='./deeplake/', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype     shape     dtype  compression
  -------    -------   -------   -------  ------- 
   text       text      (6, 1)     str     None   
 metadata     json      (6, 1)     str     None   
 embedding  embedding  (6, 768)  float32   None   
    id        text      (6, 1)     str     None   
Convert the temperature from Celsius to Fahrenheit

Input: 40°C
Output: 104°F

Input: 40°C
Output:
